In [1]:
import os
import numpy as np

### (실행X : 전처리)전체 API scanning

In [ ]:
path0_dir = './0'
path1_dir = './1'
file0_list = os.listdir(path0_dir)
file1_list = os.listdir(path1_dir)
total_API=[]
for i in file0_list:
    file = './0/%s' % i
    data = open(file,'r')
    API_list = data.readlines()
    API_list = list(set(API_list))
    for i in range(len(API_list)):
        API_list[i] = (API_list[i])[:-1]
    total_API.extend(API_list)
    total_API = list(set(total_API))
for i in file1_list:
    file = './1/%s' % i
    data = open(file,'r')
    API_list = data.readlines()
    API_list = list(set(API_list))
    for i in range(len(API_list)):
        API_list[i] = (API_list[i])[:-1]
    total_API.extend(API_list)
    total_API = list(set(total_API))

num_of_total_API = np.size(total_API)
total_API = sorted(total_API)


### (실행X : 전처리) train data와 test data 처리

In [ ]:
train_data0 = np.zeros((1,dim))
train_data1 = np.zeros((1,dim))
test_data0 = np.zeros((1,dim))
test_data1 = np.zeros((1,dim))

count=0;
for i in file0_list:
    input_data= np.zeros((1,dim))
    if(count >= len(file0_list)*(0.7) ):
        file = './0/%s' % i
        data= open(file,'r')
        sample = data.readlines()
        for j in range(len(sample)):
            sample[j] = (sample[j])[:-1]
        sample = sorted(sample)
        for j in range(len(sample)):
            for k in range(len(total_API)):
                if(sample[j] == total_API[k]):
                    input_data[0][k] +=1
                    break;
        if(count==len(file0_list)*(0.7)):
            test_data0 = input_data;
        else:
            test_data0 = np.concatenate((test_data0,input_data),axis=0)
    else:
        file = './0/%s' % i
        data = open(file,'r')
        sample = data.readlines()
        for j in range(len(sample)):
            sample[j] = (sample[j])[:-1]
        sample = sorted(sample)
        for j in range(len(sample)):
            for k in range(len(total_API)):
                if(sample[j] == total_API[k]):
                    input_data[0][k] +=1
                    break;
        if(count==0):
            train_data0 = input_data;
        else:
            train_data0 = np.concatenate((train_data0,input_data),axis=0)
        
    count +=1
        
count=0;
for i in file1_list:
    input_data= np.zeros((1,dim))
    if(count >= len(file1_list)*(0.7) ):
        file = './1/%s' % i
        data= open(file,'r')
        sample = data.readlines()
        for j in range(len(sample)):
            sample[j] = (sample[j])[:-1]
        sample = sorted(sample)
        for j in range(len(sample)):
            for k in range(len(total_API)):
                if(sample[j] == total_API[k]):
                    input_data[0][k] +=1
                    break;
        if(count==len(file1_list)*(0.7)):
            test_data1 = input_data;
        else:
            test_data1 = np.concatenate((test_data1,input_data),axis=0)
    else:
        file = './1/%s' % i
        data = open(file,'r')
        sample = data.readlines()
        for j in range(len(sample)):
            sample[j] = (sample[j])[:-1]
        sample = sorted(sample)
        for j in range(len(sample)):
            for k in range(len(total_API)):
                if(sample[j] == total_API[k]):
                    input_data[0][k] +=1
                    break;
        if(count==0):
            train_data1 = input_data;
        else:
            train_data1 = np.concatenate((train_data1,input_data),axis=0)
            
    count +=1

### 전체 API 목록 및 개수, train data와 test data 불러오기

In [2]:
opSet = np.load('./opSet.npz')
num_of_total_API = opSet['num_of_total_API']
total_API= opSet['total_API']

array = np.load("./dataset.npz")
train_data0 = array['train_data0']
train_data1 = array['train_data1']
test_data0 = array['test_data0']
test_data1 = array['test_data1']

### train data에서 TF 및 TF_IDF값 구하기

In [3]:
train0TF = np.zeros(np.shape(train_data0))
train1TF = np.zeros(np.shape(train_data1))

#### TF = log( 1 + w/d)

In [4]:
for i in range(np.size(train0TF,0)):
    for j in range(np.size(train0TF,1)):
        train0TF[i][j] = np.log(1+ (train_data0[i][j]/num_of_total_API))
for i in range( np.size(train1TF,0) ) :
    for j in range(np.size(train1TF,1)):
        train1TF[i][j] = np.log(1+ (train_data1[i][j]/num_of_total_API))      

#### IDF = 1
#### TF_IDF = TF * IDF

In [5]:
train0TF_IDF = train0TF * 1 
train1TF_IDF = train1TF * 1 

#### 각 API에 대한 평균 TF_IDF

In [6]:
train0AVG_TF_IDF = sum(train0TF_IDF)/ np.size(train0TF_IDF,0)
train1AVG_TF_IDF = sum(train1TF_IDF) / np.size(train1TF_IDF,0)

### train data에 대한 악성코드 분류
#### by cosine similarity

In [7]:
answer0_num=0;
answer1_num=0;
alpha = 0.0000000001 ## zero devide error를 막기위한 값
for i in range(np.size(train0TF_IDF,0)):
    data = train0TF_IDF[i]
    similariry0 = np.dot(data,train0AVG_TF_IDF)/np.sqrt(np.dot(data,data) * np.dot(train0AVG_TF_IDF,train0AVG_TF_IDF)+ alpha)
    similariry1 = np.dot(data,train1AVG_TF_IDF)/np.sqrt(np.dot(data,data) * np.dot(train1AVG_TF_IDF,train1AVG_TF_IDF)+ alpha)
    if(similariry0 >= similariry1):
        answer0_num +=1
        
for i in range(np.size(train1TF_IDF,0)):
    data = train1TF_IDF[i]
    similariry0 = np.dot(data,train0AVG_TF_IDF)/np.sqrt(np.dot(data,data) * np.dot(train0AVG_TF_IDF,train0AVG_TF_IDF)+ alpha)
    similariry1 = np.dot(data,train1AVG_TF_IDF)/np.sqrt(np.dot(data,data) * np.dot(train1AVG_TF_IDF,train1AVG_TF_IDF)+ alpha)
    if(similariry0 <= similariry1):
        answer1_num +=1

In [8]:
print('%0.2f' %float(answer0_num/np.size(train_data0,0) * 100) + '%')
print('%0.2f' %float(answer1_num/np.size(train_data1,0) * 100) + '%')

80.62%
53.69%


### test data에서 TF_IDF값 구하기

In [9]:
test0TF = np.zeros(np.shape(test_data0))
test1TF = np.zeros(np.shape(test_data1))

In [10]:
for i in range(np.size(test_data0,0)):
    for j in range(np.size(test_data0,1)):
        test0TF[i][j] = np.log(1+ (test_data0[i][j]/num_of_total_API))
for i in range(np.size(test_data1,0)):
    for j in range(np.size(test_data1,1)):
        test1TF[i][j] = np.log(1+ (test_data1[i][j]/num_of_total_API))       

In [11]:
test0TF_IDF = test0TF
test1TF_IDF = test1TF

### test data에 대한 악성코드 분류

In [12]:
answer0_num=0;
answer1_num=0;
test0_y = np.zeros(np.size(test0TF_IDF))
test1_y = np.zeros(np.size(test1TF_IDF))
for i in range(np.size(test0TF_IDF,0)):
    data = test0TF_IDF[i]
    similariry0 = np.dot(data,train0AVG_TF_IDF)/np.sqrt(  np.dot(data,data) * np.dot(train0AVG_TF_IDF,train0AVG_TF_IDF)+alpha )
    similariry1 = np.dot(data,train1AVG_TF_IDF)/np.sqrt(   np.dot(data,data) * np.dot(train1AVG_TF_IDF,train1AVG_TF_IDF)+alpha )
    if(similariry0 >= similariry1):
        answer0_num +=1
        test0_y[i]=0
    else:
        test0_y[i]=1
        
for i in range(np.size(test1TF_IDF,0)):
    data = test1TF_IDF[i]
    similariry0 = np.dot(data,train0AVG_TF_IDF)/np.sqrt(  np.dot(data,data) * np.dot(train0AVG_TF_IDF,train0AVG_TF_IDF)+alpha )
    similariry1 = np.dot(data,train1AVG_TF_IDF)/np.sqrt(   np.dot(data,data) * np.dot(train1AVG_TF_IDF,train1AVG_TF_IDF)+alpha )
    if(similariry0 <= similariry1):
        answer1_num +=1
        test1_y[i] =1
    else:
        test1_y[i] =0

In [13]:
print('%0.2f' %float(answer0_num /np.size(test0TF_IDF,0) * 100) + '%')
print('%0.2f' %float(answer1_num /np.size(test1TF_IDF,0) * 100) + '%')

83.30%
54.17%
